In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import svm
import numpy as np
import pandas as pd
from sklearn import preprocessing
from pandas import DataFrame
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF
from sklearn.model_selection import KFold
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr
from matplotlib.ticker import MultipleLocator, FormatStrFormatter
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from gplearn import genetic
dataset= r'0107.xlsx'

data=pd.DataFrame(pd.read_excel(dataset))
data

,A nH,A nN,A nS,A Si,A Mv,A Mi,A ETA_dAlpha_B,A ETA_BetaP,A FMF,A HOMO,...,B nS,B Mv,B Mse,B ETA_dAlpha_B,B ETA_BetaP,B FMF,B HOMO,B Bandgap,Time (s),Potential (v)
0,24,8.0,1.000000e-07,406.647744,0.633314,7.672599,0.02759,1.46552,0.415094,-4.257,...,4.0,0.728985,1.062272,0.03222,1.45833,0.428571,-5.259,-5.687571,15,0.2467
1,24,8.0,1.000000e-07,406.647744,0.633314,7.672599,0.02759,1.46552,0.415094,-4.257,...,4.0,0.728985,1.062272,0.03222,1.45833,0.428571,-5.259,-5.687571,35,0.2481
2,24,8.0,1.000000e-07,406.647744,0.633314,7.672599,0.02759,1.46552,0.415094,-4.257,...,4.0,0.728985,1.062272,0.03222,1.45833,0.428571,-5.259,-5.687571,55,0.2380
3,24,8.0,1.000000e-07,406.647744,0.633314,7.672599,0.02759,1.46552,0.415094,-4.257,...,4.0,0.728985,1.062272,0.03222,1.45833,0.428571,-5.259,-5.687571,75,0.2292
4,24,8.0,1.000000e-07,406.647744,0.633314,7.672599,0.02759,1.46552,0.415094,-4.257,...,4.0,0.728985,1.062272,0.03222,1.45833,0.428571,-5.259,-5.687571,95,0.2200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,12,2.0,1.000000e+00,261.824267,0.708947,7.480693,0.02319,1.53261,0.514286,-5.107,...,4.0,0.753763,1.051243,0.02396,1.53125,0.528736,-5.040,-5.568736,15,0.2517
386,12,2.0,1.000000e+00,261.824267,0.708947,7.480693,0.02319,1.53261,0.514286,-5.107,...,4.0,0.753763,1.051243,0.02396,1.53125,0.528736,-5.040,-5.568736,35,0.2537
387,12,2.0,1.000000e+00,261.824267,0.708947,7.480693,0.02319,1.53261,0.514286,-5.107,...,4.0,0.753763,1.051243,0.02396,1.53125,0.528736,-5.040,-5.568736,55,0.2463
388,12,2.0,1.000000e+00,261.824267,0.708947,7.480693,0.02319,1.53261,0.514286,-5.107,...,4.0,0.753763,1.051243,0.02396,1.53125,0.528736,-5.040,-5.568736,75,0.2390


In [2]:
X = data.values[:,:-1]
y = data.values[:,-1]
for i in range(X.shape[1]):
    X[:,[i]] = preprocessing.MinMaxScaler().fit_transform(X[:,[i]])

In [3]:
est_gp = genetic.SymbolicTransformer(population_size=1000,
                           generations=91, stopping_criteria=0.01,
                           p_crossover=0.8, p_subtree_mutation=0.05,
                           p_hoist_mutation=0.05, p_point_mutation=0.05,
                           max_samples=0.9, verbose=1,
                           parsimony_coefficient=0.01, random_state=None,n_components=100)

In [4]:
V=est_gp.fit(X, y)
V

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    26.65         0.106253        3         0.381914         0.314382      1.14m


SymbolicTransformer(generations=91, max_samples=0.9, n_components=100,
                    p_crossover=0.8, p_hoist_mutation=0.05,
                    p_point_mutation=0.05, p_subtree_mutation=0.05,
                    parsimony_coefficient=0.01, stopping_criteria=0.01,
                    verbose=1)

In [5]:
px=V.transform(X)
for i in range(0,50):
  pear=np.corrcoef(px[:,i], y)
  pea=pear[0,1]
  if pea>0.32:
   print(pea,i,V[i])

0.36292942149816887 4 add(mul(mul(div(X3, X2), add(X15, X6)), add(mul(0.780, X20), sub(X19, X3))), add(add(div(X13, X18), div(X6, -0.489)), div(add(X12, X20), sub(X0, X0))))
0.3392607774338739 11 div(mul(X19, X0), X2)
0.34904078787698506 12 sub(mul(mul(X8, X17), X12), X1)
0.33873266007876357 14 sub(X8, X7)
0.33028404654610916 16 div(X8, X7)
0.33873266007876357 18 sub(X8, X7)
0.33913686199079146 19 mul(div(mul(mul(sub(X7, X16), sub(X3, X12)), sub(mul(X3, X11), sub(X20, X12))), div(div(mul(X13, X8), sub(X20, X18)), add(add(X17, X12), add(X21, X7)))), add(mul(add(div(X9, X15), add(X6, X13)), mul(sub(X17, X1), div(X0, X4))), sub(mul(sub(X9, X20), mul(X22, X9)), sub(add(X17, X16), add(X12, X9)))))
0.3401034512898762 22 sub(sub(sub(mul(add(X21, X15), sub(-0.178, X17)), div(sub(X12, X13), div(X4, X22))), sub(add(add(X15, X15), sub(X16, X19)), add(mul(X6, X18), add(X16, 0.499)))), add(sub(mul(add(X22, X6), sub(X7, X6)), add(add(X5, X22), add(X20, X12))), mul(add(sub(X15, X21), div(X6, X5)), mu

In [6]:
for i in range(0,(data.shape[1]-1)):
 for j in range(0,(data.shape[1]-1)):
  for k in range(0,(data.shape[1]-1)):
    for n in range(0,(data.shape[1]-1)):
     px=(X[:,i]-X[:,j])*(X[:,k]-X[:,n])
     per=np.corrcoef(px, y)
     if per[0,1]>0.43 or per[0,1]<-0.43:
      print(per[0,1])
      print(i,j,k,n)
      print(data.columns.values.tolist()[i],data.columns.values.tolist()[j],data.columns.values.tolist()[k],data.columns.values.tolist()[n])
      print('(',data.columns.values.tolist()[i],'-',data.columns.values.tolist()[j],')','*','(',data.columns.values.tolist()[k],'-',data.columns.values.tolist()[n],')')
      print('**********************************************')  

D:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2829: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
D:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2830: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


-0.4323155223757824
1 13 4 13
A nN B nN A Mv B nN
( A nN - B nN ) * ( A Mv - B nN )
**********************************************
0.4323155223757824
1 13 13 4
A nN B nN B nN A Mv
( A nN - B nN ) * ( B nN - A Mv )
**********************************************
-0.4323155223757824
4 13 1 13
A Mv B nN A nN B nN
( A Mv - B nN ) * ( A nN - B nN )
**********************************************
0.4323155223757824
4 13 13 1
A Mv B nN B nN A nN
( A Mv - B nN ) * ( B nN - A nN )
**********************************************
0.4323155223757824
13 1 4 13
B nN A nN A Mv B nN
( B nN - A nN ) * ( A Mv - B nN )
**********************************************
-0.4323155223757824
13 1 13 4
B nN A nN B nN A Mv
( B nN - A nN ) * ( B nN - A Mv )
**********************************************
0.4323155223757824
13 4 1 13
B nN A Mv A nN B nN
( B nN - A Mv ) * ( A nN - B nN )
**********************************************
-0.4323155223757824
13 4 13 1
B nN A Mv B nN A nN
( B nN - A Mv ) * ( B nN - A nN )
***

In [7]:
function_set = ['add', 'sub', 'mul', 'div', 'log', 'sqrt', 'abs', 'neg','inv','sin','cos','tan', 'max', 'min']

est_gp = genetic.SymbolicTransformer(population_size=1000,
                           generations=91, stopping_criteria=0.01,
                           p_crossover=0.8, p_subtree_mutation=0.05,
                           p_hoist_mutation=0.05, p_point_mutation=0.05,
                           max_samples=0.9, verbose=1,function_set=function_set,
                           parsimony_coefficient=0.01, random_state=None,n_components=100)

V=est_gp.fit(X, y)
V

    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0     9.23         0.109848        3         0.414271         0.105445     52.88s


SymbolicTransformer(function_set=['add', 'sub', 'mul', 'div', 'log', 'sqrt',
                                  'abs', 'neg', 'inv', 'sin', 'cos', 'tan',
                                  'max', 'min'],
                    generations=91, max_samples=0.9, n_components=100,
                    p_crossover=0.8, p_hoist_mutation=0.05,
                    p_point_mutation=0.05, p_subtree_mutation=0.05,
                    parsimony_coefficient=0.01, stopping_criteria=0.01,
                    verbose=1)

In [8]:
px=V.transform(X)
for i in range(0,50):
  pear=np.corrcoef(px[:,i], y)
  pea=pear[0,1]
  if pea>0.32:
   print(pea,i,V[i])

0.35544295489312233 5 neg(neg(mul(max(abs(sqrt(X0)), tan(tan(X4))), cos(add(add(X7, X9), cos(X11))))))
0.36354501740843737 10 div(sqrt(inv(sin(min(X18, X19)))), add(abs(inv(log(X10))), min(sqrt(add(X11, X13)), div(sub(X10, X6), min(X4, X4)))))
0.3273528236159502 12 log(cos(sin(min(X3, X2))))
0.3510370735601932 18 neg(tan(tan(X2)))
0.3329308296544863 21 mul(sub(X8, X7), cos(X20))
0.32442771975189344 22 tan(cos(X1))
0.33174557410805805 29 abs(cos(X2))
0.32625950687967775 30 neg(sub(X3, X20))


In [9]:
for i in range(0,(data.shape[1]-1)):
 for j in range(0,(data.shape[1]-1)):
  for k in range(0,(data.shape[1]-1)):
    for n in range(0,(data.shape[1]-1)):
     px=(X[:,i]-X[:,j])*(X[:,k]-X[:,n])
     per=np.corrcoef(px, y)
     if per[0,1]>0.43 or per[0,1]<-0.43:
      print(per[0,1])
      print(i,j,k,n)
      print(data.columns.values.tolist()[i],data.columns.values.tolist()[j],data.columns.values.tolist()[k],data.columns.values.tolist()[n])
      print('(',data.columns.values.tolist()[i],'-',data.columns.values.tolist()[j],')','*','(',data.columns.values.tolist()[k],'-',data.columns.values.tolist()[n],')')
      print('**********************************************')  

D:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2829: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
D:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:2830: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


-0.4323155223757824
1 13 4 13
A nN B nN A Mv B nN
( A nN - B nN ) * ( A Mv - B nN )
**********************************************
0.4323155223757824
1 13 13 4
A nN B nN B nN A Mv
( A nN - B nN ) * ( B nN - A Mv )
**********************************************
-0.4323155223757824
4 13 1 13
A Mv B nN A nN B nN
( A Mv - B nN ) * ( A nN - B nN )
**********************************************
0.4323155223757824
4 13 13 1
A Mv B nN B nN A nN
( A Mv - B nN ) * ( B nN - A nN )
**********************************************
0.4323155223757824
13 1 4 13
B nN A nN A Mv B nN
( B nN - A nN ) * ( A Mv - B nN )
**********************************************
-0.4323155223757824
13 1 13 4
B nN A nN B nN A Mv
( B nN - A nN ) * ( B nN - A Mv )
**********************************************
0.4323155223757824
13 4 1 13
B nN A Mv A nN B nN
( B nN - A Mv ) * ( A nN - B nN )
**********************************************
-0.4323155223757824
13 4 13 1
B nN A Mv B nN A nN
( B nN - A Mv ) * ( B nN - A nN )
***